In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import os
import cv2
from sklearn.model_selection import train_test_split


In [7]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch
from efficientnet_pytorch import EfficientNet

# Paths setup
data_dir = r"C:\Users\DELL\Desktop\Datasets\ForClassification\wfd\wfd_dataset"
csv_dir = r"C:\Users\DELL\Desktop\Datasets\ForClassification\wfd"
train_file = os.path.join(csv_dir, 'data_train.csv')
test_file = os.path.join(csv_dir, 'data_test.csv')
valid_file = os.path.join(csv_dir, 'data_valid.csv')

class WheatDiseaseDataset(Dataset):
    def __init__(self, csv_file, data_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.data_dir, self.data.iloc[idx]['image_name'])
        image = Image.open(img_name).convert("RGB")
        labels = torch.tensor(self.data.iloc[idx][1:].values.astype(float))  # Multi-label target

        if self.transform:
            image = self.transform(image)

        return image, labels


In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, ShiftScaleRotate, RandomBrightnessContrast, Cutout
)
from albumentations.pytorch import ToTensorV2
import albumentations as A

def get_transforms():
    return A.Compose([
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
        Cutout(num_holes=8, max_h_size=16, max_w_size=16, p=0.5),
        A.Resize(512, 512),
        ToTensorV2()
    ])


In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load EfficientNet Model
model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=5)  # Assuming 5 diseases
model = model.to(device)

# Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-6)
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

# DataLoader
train_dataset = WheatDiseaseDataset(train_file, data_dir, transform=get_transforms())
valid_dataset = WheatDiseaseDataset(valid_file, data_dir, transform=get_transforms())
test_dataset = WheatDiseaseDataset(test_file, data_dir, transform=get_transforms())

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
from tqdm import tqdm

def train_model(model, train_loader, valid_loader, criterion, optimizer, scheduler, num_epochs=50):
    best_acc = 0.0

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()

        # Validation phase
        model.eval()
        val_acc = 0.0
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                predictions = (torch.sigmoid(outputs) > 0.5).float()
                val_acc += (predictions == labels).sum().item() / labels.numel()

        val_acc /= len(valid_loader)

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Val Acc: {val_acc:.4f}")
        scheduler.step()


In [ ]:
from sklearn.metrics import classification_report

def evaluate_model(model, test_loader):
    model.load_state_dict(torch.load("best_model.pth"))
    model.eval()
    
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            predictions = (torch.sigmoid(outputs) > 0.5).cpu().numpy()
            all_predictions.extend(predictions)
            all_labels.extend(labels.numpy())

    print(classification_report(all_labels, all_predictions))
